## Artificial Neural Network built with Tensorlow

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

# Problem Statement: Can we predict if our customers will leave our bank? If we can predict that a customer might leave, we can send them promotions to keep them happy.

## Part 1 - Data Preprocessing

Import Dataset
- Here we remove the first three columns: row number, customerid, and surname because they will not have an effect on our outcome
- 1 is left the bank, 0 is stayed with the bank

In [2]:
dataset = pd.read_csv("Churn_Modelling.csv")
x = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding Categorical Data

Label Encoding the "Gender" Column

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,2] = le.fit_transform(x[:,2])

In [7]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One hot encoding the "geography" column

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder="passthrough")
x = np.array(ct.fit_transform(x))

In [9]:
x

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

### Split Dataset into Train and Test

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=0)

### Feature Scaling
- Only fitted to the training set to avoid information leakage

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [12]:
ann = tf.keras.models.Sequential()
# Use sequential so that it builds our model as a sequence of layers

### Adding the input layer and first hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units = 6, activation='relu'))
# Dense first paramter is unit, which is asking how many neurons you want, there is no rule of thumb
# its all trial and error for your specific data
# Use somethng logical
# Then we want to use the rectifier activation, which is relu
# Why rectifier? The activation function in the hidden layers of a fully connected neural network must be rectifier

### Adding the second hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units = 6, activation='relu'))
# all you have to do to add a new layer is copy and paste the first hidden layer
# You can change the neurons and hyperparameter values in the layers if you want!

### Adding the output layer

In [15]:
ann.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))
# We are still adding a new layer, but we change the parameters
# We change 6 to 1, because we are predicting a binary variable
# However, if we were predicting 3 dependent variables like A,B,C - we would need 3 output layers
# The activation function is changed to a sigmoid activation function
# Why? It gives you the predictions and the probability that the outcome is 1
# For non-binary classification, the activation should be softmax not sigmoid

## Part 3 - Training the ANN

### Compiling the ANN

In [16]:
ann.compile(optimizer = "adam" , loss = "binary_crossentropy" , metrics = ["accuracy"] )
# the best optimizer is adam, why? because it is the best stochastic gradient descent optimizer
# loss function must always match the dependent variable
#           meaning since we have a binary y we are predicting, the loss must be binary
# For non-binary classification, we would use categorical_crossentropy
# we can enter multiple metrics at once

### Training the ANN on the training set

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)
# fit method is always used for training
# we have to specify the batch size, batch size increases our speed and efficiency. 32 is the default value
# an epoch is one full training of our model, how many times do we want it to do it?

Epoch 1/100
250/250 [==============================] - 0s 589us/step - loss: 0.5277 - accuracy: 0.7784
Epoch 2/100
250/250 [==============================] - 0s 558us/step - loss: 0.4589 - accuracy: 0.7987
Epoch 3/100
250/250 [==============================] - 0s 580us/step - loss: 0.4322 - accuracy: 0.8123
Epoch 4/100
250/250 [==============================] - 0s 585us/step - loss: 0.4132 - accuracy: 0.8196
Epoch 5/100
250/250 [==============================] - 0s 580us/step - loss: 0.3955 - accuracy: 0.8330
Epoch 6/100
250/250 [==============================] - 0s 580us/step - loss: 0.3803 - accuracy: 0.8411
Epoch 7/100
250/250 [==============================] - 0s 508us/step - loss: 0.3695 - accuracy: 0.8444
Epoch 8/100
250/250 [==============================] - 0s 512us/step - loss: 0.3627 - accuracy: 0.8482
Epoch 9/100
250/250 [==============================] - 0s 508us/step - loss: 0.3579 - accuracy: 0.8520
Epoch 10/100
250/250 [==============================] - 0s 508us/step - l

## Part 4 - Making the Predictions and evaluating the model

### Predicting a single customer
Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer ?

### Solution

In [18]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
# We have to apply the transform function the on observations
# Since we chose a sigmoid activation function, we have to compare it against greater than 0.5

[[False]]


In [19]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
# They have a 0.03 predicted chance of leaving our bank

[[0.03487313]]


### Since we got false, this customer is not predicted to leave our bank

### Predicting the Test Results

In [20]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
# Left is predicted, right is actual

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the confusion matrix

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1506   89]
 [ 189  216]]


0.861

86% Accuracy
- 1506 - Correct Predictions that the customer stays with the bank
- 216 - Correct Predictions that the customer leaves the bank
- 89 - Incorrect Predictions that the customer would leave the bank
- 189 - Incorrect Predictions that the customer would stay with the bank.